# load packages

In [1]:
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, lemmatize
from gensim.models import CoherenceModel
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.models import LsiModel

nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools 
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inline

# Enable logging for gensim - optional
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
#logging.root.setLevel(level=logging.INFO)
#import warnings
#warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [2]:
# Load dataset
df = pd.read_csv('C:/Users/Kornelius/Desktop/Data 2/nips-papers/papers.csv', header = 0, sep = ';', error_bad_lines=False)
# Drop the columns not needed
df = df.drop(columns=['id', 'event_type', 'pdf_name'], axis=1)

In [3]:
# Drop a row by condition
df = df[df.abstract != 'Abstract Missing']

In [4]:
data = df.abstract.values.tolist()

# Pre-processing (Baseline)

In [5]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(data):
    stop_free = " ".join([i for i in data.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

data = [clean(data).split() for data in data] 

In [6]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [7]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(16129 unique tokens: ['activity', 'also', 'analysis', 'applied', 'assume']...)


In [8]:
id2word.filter_n_most_frequent(10)
print((id2word))

Dictionary(16119 unique tokens: ['activity', 'also', 'analysis', 'applied', 'assume']...)


In [9]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

# LSI

In [10]:
# Build the LSI Model
lsi_model = LsiModel(corpus=corpus, id2word=id2word, num_topics=1, decay=0.5)

lsi_model5 = LsiModel(corpus=corpus, id2word=id2word, num_topics=5, decay=0.5)

lsi_model10 = LsiModel(corpus=corpus, id2word=id2word, num_topics=10, decay=0.5)

lsi_model20 = LsiModel(corpus=corpus, id2word=id2word, num_topics=20, decay=0.5)

lsi_model50 = LsiModel(corpus=corpus, id2word=id2word, num_topics=50, decay=0.5)

lsi_model100 = LsiModel(corpus=corpus, id2word=id2word, num_topics=100, decay=0.5)



In [11]:
# View Topics
pprint(lsi_model.print_topics(-1))

# Print the Keyword in the 5 topics
pprint(lsi_model5.print_topics(-1))
doc_lsi = lsi_model5[corpus]

# Print the Keyword in the 5 topics
pprint(lsi_model10.print_topics(-1))
doc_lsi = lsi_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lsi_model20.print_topics(-1))
doc_lsi = lsi_model20[corpus]

# Print the Keyword in the 20 topics
pprint(lsi_model50.print_topics(-1))
doc_lsi = lsi_model50[corpus]

# Print the Keyword in the 40 topics
pprint(lsi_model100.print_topics(-1))
doc_lsi = lsi_model100[corpus]

[(0,
  '0.213*"network" + 0.204*"function" + 0.145*"using" + 0.134*"feature" + '
  '0.132*"distribution" + 0.129*"task" + 0.126*"propose" + 0.123*"image" + '
  '0.121*"based" + 0.115*"new"')]
[(0,
  '0.213*"network" + 0.204*"function" + 0.145*"using" + 0.134*"feature" + '
  '0.132*"distribution" + 0.129*"task" + 0.126*"propose" + 0.123*"image" + '
  '0.121*"based" + 0.115*"new"'),
 (1,
  '0.611*"network" + 0.289*"image" + -0.265*"function" + 0.243*"neural" + '
  '-0.182*"bound" + 0.155*"deep" + 0.108*"object" + -0.103*"matrix" + '
  '0.102*"training" + 0.101*"task"'),
 (2,
  '0.541*"network" + -0.457*"image" + -0.301*"feature" + 0.263*"function" + '
  '-0.223*"object" + 0.157*"neural" + -0.140*"task" + -0.103*"visual" + '
  '0.102*"gradient" + 0.100*"bound"'),
 (3,
  '0.615*"function" + -0.291*"distribution" + 0.255*"image" + '
  '-0.243*"inference" + 0.168*"loss" + 0.146*"feature" + -0.139*"matrix" + '
  '0.114*"optimization" + -0.114*"sample" + -0.103*"latent"'),
 (4,
  '-0.512*"matr

[(0,
  '0.213*"network" + 0.204*"function" + 0.145*"using" + 0.134*"feature" + '
  '0.132*"distribution" + 0.129*"task" + 0.126*"propose" + 0.123*"image" + '
  '0.121*"based" + 0.115*"new"'),
 (1,
  '-0.611*"network" + -0.289*"image" + 0.265*"function" + -0.243*"neural" + '
  '0.182*"bound" + -0.155*"deep" + -0.108*"object" + 0.103*"matrix" + '
  '-0.102*"training" + -0.101*"task"'),
 (2,
  '0.541*"network" + -0.457*"image" + -0.301*"feature" + 0.263*"function" + '
  '-0.223*"object" + 0.156*"neural" + -0.141*"task" + -0.103*"visual" + '
  '0.102*"gradient" + 0.100*"bound"'),
 (3,
  '-0.614*"function" + 0.291*"distribution" + -0.255*"image" + '
  '0.243*"inference" + -0.168*"loss" + -0.146*"feature" + 0.140*"matrix" + '
  '-0.116*"optimization" + 0.112*"sample" + 0.103*"latent"'),
 (4,
  '0.513*"matrix" + -0.377*"distribution" + 0.276*"feature" + '
  '-0.260*"inference" + -0.222*"function" + -0.179*"image" + 0.144*"kernel" + '
  '0.132*"network" + 0.117*"sparse" + -0.114*"process"'),
 

# Evaluation

In [12]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Perplexity not working?
#print('\nPerplexity: ', lsi_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



coherence_model_lsa = CoherenceModel(model=lsi_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score5: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score10: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score20: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score50: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score100: ', coherence_lsi)


coherence_model_lsa = CoherenceModel(model=lsi_model, texts=texts, dictionary=id2word, coherence='u_mass')
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score5: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score10: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score20: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score50: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score100: ', coherence_lsi)


Coherence Score:  0.22281864002283341

Coherence Score5:  0.3806249962175456

Coherence Score10:  0.3869856702390688

Coherence Score20:  0.31704672826511204

Coherence Score50:  0.252665354251505

Coherence Score100:  0.221193508455074

Coherence Score:  -1.690559377469005

Coherence Score5:  -2.256508129738495

Coherence Score10:  -2.266968287567413

Coherence Score20:  -2.3504710458741433

Coherence Score50:  -2.4136645622212667

Coherence Score100:  -2.548247866006441
